# **Installing necessary Libraries**

In [1]:
!pip install -q -U transformers torchtext datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 96.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.5 MB/s eta 0:00:00


In [2]:
!pip install -q -U watermark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.9 MB/s eta 0:00:00


In [3]:
%load_ext watermark
%watermark -a "Drishti Sharma" -v -p torch,torchtext,transformers

Author: Drishti Sharma

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 7.34.0

torch       : 2.0.1+cu118
torchtext   : 0.15.2
transformers: 4.33.2



In [4]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"


In [5]:
!pip install -q ipython-autotime
%load_ext autotime

time: 256 µs (started: 2023-09-25 16:35:24 +00:00)


## **Importing Essential Libraries**

In [6]:
import pandas as pd
import requests
import torch
import torch.nn.functional as F

import transformers
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, AdamW

from datasets import load_dataset, DatasetDict

time: 2.29 s (started: 2023-09-25 16:35:24 +00:00)


# **General Settings**

In [7]:
torch.backends.cudnn.deterministic = True
RANDOM_SEED = 42
torch.manual_seed(RANDOM_SEED)
DEVICE = "cpu"
#torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_EPOCHS = 1

time: 3.82 ms (started: 2023-09-25 16:35:26 +00:00)


# **Download Dataset**

In [8]:
dataset = load_dataset("sst2")

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

time: 8.25 s (started: 2023-09-25 16:35:26 +00:00)


In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 872
    })
    test: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 1821
    })
})

time: 2.4 ms (started: 2023-09-25 16:35:34 +00:00)


In [10]:
print("Unique labels in training set:", set(dataset["train"]["label"]))
print("Unique labels in validation set:", set(dataset["validation"]["label"]))
print("Unique labels in test set:", set(dataset["test"]["label"]))


Unique labels in training set: {0, 1}
Unique labels in validation set: {0, 1}
Unique labels in test set: {-1}
time: 28.7 ms (started: 2023-09-25 16:35:34 +00:00)


# Converting Dataset to Pandas DataFrame

In [11]:
# Convert each dataset to a Pandas DataFrame
train_df = pd.DataFrame(dataset["train"])
valid_df = pd.DataFrame(dataset["validation"])
test_df = pd.DataFrame(dataset["test"])

time: 2.47 s (started: 2023-09-25 16:35:34 +00:00)


In [12]:
train_df

,idx,sentence,label
0,0,hide new secretions from the parental units,0
1,1,"contains no wit , only labored gags",0
2,2,that loves its characters and communicates som...,1
3,3,remains utterly satisfied to remain the same t...,0
4,4,on the worst revenge-of-the-nerds clichés the ...,0
...,...,...,...
67344,67344,a delightful comedy,1
67345,67345,"anguish , anger and frustration",0
67346,67346,"at achieving the modest , crowd-pleasing goals...",1
67347,67347,a patient viewer,1


time: 19.7 ms (started: 2023-09-25 16:35:37 +00:00)


# **Preprocessing Dataset**

In [13]:
# Filter out rows in the test set where label is -1 since it is not present in train and val set ---> was causing error.
test_df = test_df[test_df["label"] != -1]

time: 1.16 ms (started: 2023-09-25 16:35:37 +00:00)


In [14]:
# Utilizing original val set as test set and obtaining new val set from train set
from sklearn.model_selection import train_test_split

# Split the original training data into a new training set and a validation set
train_texts, new_valid_texts, train_labels, new_valid_labels = train_test_split(
    train_df["sentence"], train_df["label"], test_size=0.1, random_state=42
)



time: 731 ms (started: 2023-09-25 16:35:37 +00:00)


In [19]:


train_texts = train_texts.reset_index(drop=True)
train_labels = train_labels.reset_index(drop=True)

valid_texts = new_valid_texts.reset_index(drop=True)
valid_labels = new_valid_labels.reset_index(drop=True)



test_texts = valid_df["sentence"].reset_index(drop=True)
test_labels = valid_df["label"].reset_index(drop=True)

time: 4.46 ms (started: 2023-09-25 16:37:40 +00:00)


# **Tokenization**

In [20]:
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

time: 274 ms (started: 2023-09-25 16:37:46 +00:00)


In [21]:
train_encodings = tokenizer(list(train_texts), truncation = True, padding = True)
valid_encodings = tokenizer(list(valid_texts), truncation = True, padding = True)
test_encodings = tokenizer(list(test_texts), truncation = True, padding = True)

time: 2.4 s (started: 2023-09-25 16:37:46 +00:00)


In [22]:
train_encodings[0]

Encoding(num_tokens=66, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

time: 2.33 ms (started: 2023-09-25 16:37:48 +00:00)


In [23]:
train_encodings[1]

Encoding(num_tokens=66, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

time: 2.84 ms (started: 2023-09-25 16:37:50 +00:00)


# **Dataset Class and Loaders**

In [25]:
class AmazonDataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self, idx):
    item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    item["labels"] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.labels)

time: 630 µs (started: 2023-09-25 16:37:52 +00:00)


In [26]:
train_dataset = AmazonDataset(train_encodings, train_labels)
valid_dataset = AmazonDataset(valid_encodings, valid_labels)
test_dataset = AmazonDataset(test_encodings, test_labels)

time: 480 µs (started: 2023-09-25 16:38:57 +00:00)


In [27]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=16, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False)

time: 1.1 ms (started: 2023-09-25 16:38:57 +00:00)


# **Loading the Model**

In [28]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
model.to(DEVICE)
model.train()

optim = AdamW(model.parameters(), lr=5e-5)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


time: 2.09 s (started: 2023-09-25 16:38:59 +00:00)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


# **Compute Metrics**

In [29]:
def compute_accuracy(model, data_loader, device):

    with torch.no_grad():

        correct_pred, num_examples = 0, 0

        for batch_idx, batch in enumerate(data_loader):

            ### Prepare data
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss, logits = outputs['loss'], outputs['logits']

            _, predicted_labels = torch.max(logits, 1)

            num_examples += labels.size(0)

            correct_pred += (predicted_labels == labels).sum()
    return correct_pred.float()/num_examples * 100

time: 620 µs (started: 2023-09-25 16:39:01 +00:00)


In [30]:
import time

time: 375 µs (started: 2023-09-25 16:39:01 +00:00)


# **Traning and Evaluating the Model**

In [31]:
start_time = time.time()

for epoch in range(NUM_EPOCHS):

    model.train()
    total_loss = 0.0  # To compute average loss for the epoch

    for batch_idx, batch in enumerate(train_loader):

        ### Prepare data
        input_ids = batch['input_ids'].to(DEVICE)
        attention_mask = batch['attention_mask'].to(DEVICE)
        labels = batch['labels'].to(DEVICE)

        ### Forward
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss, logits = outputs['loss'], outputs['logits']
        total_loss += loss.item()

        ### Backward
        optim.zero_grad()
        loss.backward()
        optim.step()

    avg_train_loss = total_loss / len(train_loader)  # Compute average loss

    ### Evaluate the model after every epoch
    model.eval()

    with torch.set_grad_enabled(False):
        train_acc = compute_accuracy(model, train_loader, DEVICE)
        valid_acc = compute_accuracy(model, valid_loader, DEVICE)

        print(f'Epoch: {epoch+1:04d}/{NUM_EPOCHS:04d} | '
              f'Avg Train Loss: {avg_train_loss:.4f} | '
              f'Training Accuracy: {train_acc:.2f}% | '
              f'Validation Accuracy: {valid_acc:.2f}%')

        print(f'Time elapsed after epoch {epoch+1}: {(time.time() - start_time)/60:.2f} min')

print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')
print(f'Test accuracy: {compute_accuracy(model, test_loader, DEVICE):.2f}%')


Epoch: 0001/0001 | Avg Train Loss: 0.2254 | Training Accuracy: 96.26% | Validation Accuracy: 94.18%
Time elapsed after epoch 1: 52.96 min
Total Training Time: 52.96 min
Test accuracy: 87.73%
time: 53min 5s (started: 2023-09-25 16:39:01 +00:00)


time: 53min 5s (started: 2023-09-25 16:39:01 +00:00)
